In [2]:
#在这个教程中，可以学习如何使用数据并行(DataParallel)来使用多GPU
#导入参数，导入pytorch模块和定义参数

import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

#Parameters 和 DataLoaders
input_size = 5
output_size = 2

batch_size = 30
data_size = 100

#设备(Device)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")




In [3]:
#虚拟数据集，要制作一个虚拟（随机）数据集，只需要实现__getitem__

class RandomDataset(Dataset):
    
    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)
        
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return self.len
    
rand_loader = DataLoader(dataset=RandomDataset(input_size, data_size), batch_size=batch_size, shuffle=True)



In [5]:
class Model(nn.Module):
    #Our model
    
    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        
    def forward(self, input):
        output = self.fc(input)
        print("\t In Model: Input size",input.size(),
             "output size", output.size())
        
        return output
    
    


In [6]:
model = Model(input_size, output_size)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    
    model = nn.DataParallel(model)
    
model.to(device)






Model(
  (fc): Linear(in_features=5, out_features=2, bias=True)
)

In [7]:
for data in rand_loader:
    input = data.to(device)
    output = model(input)
    print("Outside:input size", input.size(),
         "output_size", output.size())



	 In Model: Input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside:input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	 In Model: Input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside:input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	 In Model: Input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside:input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	 In Model: Input size torch.Size([10, 5]) output size torch.Size([10, 2])
Outside:input size torch.Size([10, 5]) output_size torch.Size([10, 2])
